In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from utils.modelos import *
from utils.transformacion import *
from utils.recomendaciones import *


VAR_SEED = 42
VAR_TESTSET_SIZE = 0.20
VAR_DIR_DATA_CLEAN = '../data/cleaning'

df_dataset = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/mf_dataset.csv", sep=",", encoding="latin1")

df_catalogo = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/catalogo.csv", sep=",", encoding="latin1")[['id_ejercicio', 'hito', 'skill', 'knowledge']]
df_catalogo = calcular_ratio_interacciones(df_dataset, df_catalogo)
df_catalogo = calcular_puntuacion_dataset(
    dataframe=df_catalogo, 
    caracteristicas={"hito": (1, 4), "skill": (0, 15), "knowledge": (1, 15), "interaction_ratio": (0.0, 1.0)}, 
    pesos={"hito": 0.4, "skill": 0.25, "knowledge": 0.25, "interaction_ratio": 0.1}, 
    nueva_columna='score'
)

df_interactions = factorizacion_a_calificaciones(df_catalogo, df_dataset, columna_usuario='id_estudiante')
df_interactions[['id_ejercicio', 'hito', 'skill', 'knowledge']] = df_interactions[['id_ejercicio', 'hito', 'skill', 'knowledge']].astype(int)
df_interactions.head(16)

2024-11-12 17:16:02.555668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731442562.624747   30064 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731442562.642631   30064 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 17:16:02.797063: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,id_estudiante,id_ejercicio,hito,skill,knowledge,ratio_interaccion,score
0,0,0,1,1,7,0.787206,0.123810
1,0,1,2,1,1,0.359008,0.150000
2,0,3,1,0,3,0.707572,0.035714
3,0,4,1,0,1,0.956919,0.000000
4,0,6,2,1,3,0.361619,0.185714
5,0,10,1,0,3,0.781984,0.035714
6,0,17,1,0,2,0.962141,0.017857
7,0,18,1,0,3,0.507833,0.035714
8,0,22,1,1,1,0.848564,0.016667
9,0,23,1,0,2,0.770235,0.017857


In [2]:
recommender = RecommenderSystemDCNv2(df_interactions)
recommender.train(epochs=100, print_every=10)

/home/edgar-ramxs/sistema-de-recomendacion-unab/.sistema-de-recomendacion-unab-env/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Epoch 0, Loss: 9.5159
Epoch 10, Loss: 6.5396
Epoch 20, Loss: 4.5107
Epoch 30, Loss: 3.1638
Epoch 40, Loss: 2.2669
Epoch 50, Loss: 1.6570
Epoch 60, Loss: 1.2308
Epoch 70, Loss: 0.9252
Epoch 80, Loss: 0.7016
Epoch 90, Loss: 0.5356


In [3]:
user_id_to_recommend = 0
recommended_items = recommender.recommend_for_user(user_id=user_id_to_recommend, num_recommendations=10)
print(f"Recomendaciones para el usuario {user_id_to_recommend}: {recommended_items}")

Recomendaciones para el usuario 0: [23, 19, 30, 16, 24, 25, 15, 18, 17, 22]
